# 0. 공통 코드 부분(필요한 라이브러리 설치, 자연어 처리)

## 필요한 라이브러리 설치

In [1]:
# !pip install networkx
# !pip install gensim
# !pip install pyLDAvis
# !pip install xlsxwriter
# !pip install nltk
# !pip install numpy

In [1]:
import re
import json
import pandas as pd
import xlsxwriter
import numpy as np

### option A: 네이버 블로그 csv파일 불러올 때는 이하의 코드를 실행한 후 optionB (쇼핑후기) 코드는 건너뛰고 바로 공통 코드로 이동

In [3]:
blog_df = pd.read_csv('SK_Microwave(블로그).csv', encoding='utf-8-sig') # 네이버 블로그 csv파일 불러오기

In [4]:
blog_df = blog_df.sort_values(by='Post Date' ,ascending=True) # post Date를 기준으로 오름차순으로 정렬
blog_df['Post Date'] = [str(dates).replace('.0','') for dates in blog_df['Post Date']] # 분석을 위해서 날짜의 형식을 변경함. 

In [5]:
# 문서에 'Post Date'에서 빈칸을 nan (빈데이터)를 찾아주고 삭제해주는 과정 = 나중에 분석에 오류를 없애기 위해서
drop_index = blog_df['Post Date'][blog_df['Post Date']=='nan'].index
d = [d for d in drop_index] 
print(d, '\n') # 아래 결과 예) 751 번째 행의 날짜가 nan으로 표시 되어있음을 알 수 있음. 

blog_df.drop(d, axis=0, inplace=True) # nan 데이터가 있는 행을 삭제해줌.
print(blog_df.shape)

[] 

(570, 7)


In [7]:
# 필요한 부분만 편집해서 가져오기. 
# 블로그 데이터 파일에서 블로그 제목인 'Title'과 블로그 내용인 'Description' 및 게시일인 'Post Date'에 해당하는 부분만 가져와 저장.
tm_post = blog_df.get('Title') + " " + blog_df.get('Description')
tm_post_date = blog_df.get('Post Date')

tm_post

397    SK 매직 전자레인지 구매 후기 떡하니 매직이라고 인쇄 되어있고 뒤엔 KC마크가 듬...
414    11번가 SK매직 전자레인지 MWO-230KG 신제품 런칭기념 이벤트 오늘은 SK매...
519    [전자레인지] SK매직 전자레인지 MWO-20M7 스마트한 요리을...  스마트한 ...
92     기본 성능이 뛰어나다! SK매직 전자레인지 MWO-230KG 후기!! 음식을 데우고...
88     간편한 조리는 SK매직 전자레인지 MWO-23KG 그동안 전자레인지 없이 살아왔는데...
                             ...                        
236    개성있는 SK매직 자동요리 전자식 전자레인지 선택해하셔요 11시 1분 6초 포스팅 ...
320    모두가 추천하는 SK매직 ALL IN ONE 멀티오븐 전자레인지...  모두가 추천...
131    요즘인기인 SK매직 ALL IN ONE 멀티오븐 전자레인지 버튼...  #멀티오븐 ...
510    EONB401SA SK매직 빌트인 스팀 오븐 프리미엄 살려드립니다 SK매직 스팀 오...
503    부산정수기렌탈 SK매직의 놀라움! 부산정수기렌탈 SK매직의 놀라움! 여러분 안녕하세...
Length: 570, dtype: object

In [8]:
tm_post = tm_post.drop(index=[503], axis=0)

In [9]:
tm_post

397    SK 매직 전자레인지 구매 후기 떡하니 매직이라고 인쇄 되어있고 뒤엔 KC마크가 듬...
414    11번가 SK매직 전자레인지 MWO-230KG 신제품 런칭기념 이벤트 오늘은 SK매...
519    [전자레인지] SK매직 전자레인지 MWO-20M7 스마트한 요리을...  스마트한 ...
92     기본 성능이 뛰어나다! SK매직 전자레인지 MWO-230KG 후기!! 음식을 데우고...
88     간편한 조리는 SK매직 전자레인지 MWO-23KG 그동안 전자레인지 없이 살아왔는데...
                             ...                        
102    SK매직 고급형 전자레인지 20L MWO-20EC2 담소 SK매직 고급형 전자레인지...
236    개성있는 SK매직 자동요리 전자식 전자레인지 선택해하셔요 11시 1분 6초 포스팅 ...
320    모두가 추천하는 SK매직 ALL IN ONE 멀티오븐 전자레인지...  모두가 추천...
131    요즘인기인 SK매직 ALL IN ONE 멀티오븐 전자레인지 버튼...  #멀티오븐 ...
510    EONB401SA SK매직 빌트인 스팀 오븐 프리미엄 살려드립니다 SK매직 스팀 오...
Length: 569, dtype: object

### 공통: 여기에서부터는 optionA (블로그) 및 optionB(쇼핑후기) 공통으로 진행하는 코드

In [12]:
tm_post.head()  # 첫 5행 보여주기

0    잘 받았습니다~!배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니~ 많이 늦지는 ...
1    제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진...
2    포장 잘해서 적당히 도착 했는데저가 제풍이라고 불량제품-(표시한부분과 그위에 일자로...
3    장점: 일단 간편하고 간단하게 사용하기 좋습니다.... 장점: 일단 간편하고 간단하...
4    고장나서샀는데 크기가 아담합니다^^;;물론 리터계산을 안한것은아닌데 지금까지바꾸기전...
dtype: object

In [13]:
tm_post.tail()  # 마지막 5행 보여주기

995    쓰기좋아요 만족합니다 쓰기좋아요 만족합니다
996      빠른배송감사드립니다 빠른배송감사드립니다
997      깔끔하이생깄네요ㅎㅎ 깔끔하이생깄네요ㅎㅎ
998      빠른배송크기적당해요 빠른배송크기적당해요
999    가격대비 아주 좋네요 가격대비 아주 좋네요
dtype: object

In [14]:
tm_post_date.max()   # 최대 날짜 (가장 최근 데이터)

20211023

In [15]:
tm_post_date.min()  # 최소 날짜 (가장 오래된 데이터)

20181223

In [16]:
# tm_post 전처리

print(tm_post.isnull().values.any()) # 빈 행이 있는지 확인하기 ( 뒤에 에러가 날 수 있음 ) 
print(tm_post.isnull().sum()) # null 값이 총 몇개가 있는지 확인 

tm_post = tm_post.dropna(how = 'any') # Null 값이 존재하는 행을 제거

False
0


In [17]:
len(tm_post)

1000

In [18]:
import re

def cleanText(datas): # 단어 전처리
    data1 = re.sub('[^A-Za-z0-9가-힣;,.?!]', ' ', datas)
    data1 = data1.replace(' 가스레인지', ' 가스레인지 ') #검색어 띄어쓰기 붙여주기
    data1 = data1.replace('가스레인지 ', ' 가스레인지 ') #검색어 띄어쓰기 붙여주기
    data1 = data1.replace('  ', ' ')
    data1 = data1.replace('렌지', '레인지')
    data2 = data1.split()
    return data1

In [19]:
import re

def cleanText(datas): # 단어 전처리
    data1 = re.sub('[^A-Za-z0-9가-힣;,.?!]', ' ', datas)
    data1 = data1.replace(' 가스레인지', ' 가스레인지 ') #검색어 띄어쓰기 붙여주기
    data1 = data1.replace('가스레인지 ', ' 가스레인지 ') #검색어 띄어쓰기 붙여주기
    data1 = data1.replace('  ', ' ')
    data1 = data1.replace('렌지', '레인지')
    data2 = data1.split()
    return data1

In [20]:
tm_posts = []
for posts in tm_post:
    post = cleanText(posts)
    post1 = ' '.join(post)
    tm_posts.append(post)
    
tm_posts[:3]

['잘 받았습니다 !배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니 많이 늦지는 않았습니다 그런데 배송 받고 보니...사진처럼 부딪히고 찢어진 것들이 보여서 !안의 제품에 문제는 잘 받았습니다 ! 배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니 많이 늦지는 않았습니다  그런데 배송 받고 보니... 사진처럼 부딪히고 찢어진 것들이 보여서 ! 안의 제품에 문제는 없나 생각했는데 ! 오늘 만두를 넣고 돌려보니 아주 잘 됩니다   안은 스티로폼으로 쌓여 있어서 제품이 멀쩡했습니다  충격은 좀 받았을 것도 같습니다. 좀 더 조심히 배송이나 관리되면 좋겠습니다. 인터넷에 찾다가 여기가 가격이 싸기에 얼른 구입했고, 좋습니다 !  ! 참, 제조일자가 2020년 2월입니다 !  ',
 '제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진 멀티제품으로 생각이 바뀌게되어... 반품 하려고했으니 일주일이 지나버렸네요..지인한테 넘기던지 제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진 멀티제품으로 생각이 바뀌게되어... 반품 하려고했으니 일주일이 지나버렸네요..지인한테 넘기던지 하고 멀티제품으로 가야할것같네요 하지만 이가격에 사용편리하고 제품외관까지 생각해주셔서 감사합니닷   ..멀티제품도 sk매직으로 생각하고 있어요 ',
 '포장 잘해서 적당히 도착 했는데저가 제풍이라고 불량제품 표시한부분과 그위에 일자로 길게, 긴 부분은 사진으론 안보여 촬영 포기 녹이난거 같은데 그냥 동장해서 조립 판매...이건 포장 잘해서 적당히 도착 했는데 저가 제풍이라고 불량제품 표시한부분과 그위에 일자로 길게, 긴 부분은 사진으론 안보여 촬영 포기 녹이난거 같은데 그냥 동장해서 조립 판매...이건 아닌거 같은데 하면서도 저가니까 그냥 쓰자니찝찝하고 그래도 아쉬우니까 써야죠']

In [21]:
# 형태소 분석기 Mecab 불러오기

from konlpy.tag import Mecab

tagger = Mecab(dicpath='C:/mecab/mecab-ko-dic')

In [22]:
# 각자 데이터에 맞는 불용어를 추가하여 불용어 처리.
tm_stop_words = "게 점 자 수 후 저 쪽 음 은 는 이 가 배 거 츠 의 앞 강 글 애 미 번 것 등 더 를 좀 즉 인 옹 때 만 원 이때 개 일 기 시 트 드 기 탭"
tm_stop_words = tm_stop_words.split(' ')
print(tm_stop_words)

['게', '점', '자', '수', '후', '저', '쪽', '음', '은', '는', '이', '가', '배', '거', '츠', '의', '앞', '강', '글', '애', '미', '번', '것', '등', '더', '를', '좀', '즉', '인', '옹', '때', '만', '원', '이때', '개', '일', '기', '시', '트', '드', '기', '탭']


In [23]:
# 불용어를 제외한 텍스트의 형태소 분석 수행

words = []
for post in tm_posts:
    words.extend(tagger.pos(post))
    
words[0:10]

[('잘', 'MAG'),
 ('받', 'VV'),
 ('았', 'EP'),
 ('습니다', 'EF'),
 ('!', 'SF'),
 ('배송', 'NNG'),
 ('은', 'JX'),
 ('빠르', 'VA'),
 ('지', 'EC'),
 ('않', 'VX')]

In [24]:
tm_nouns = []   # 명사 추출하기
for post in tm_posts:
        if type(post) == str:
            for noun in tagger.nouns(post):
                if noun not in tm_stop_words:
                    tm_nouns.append(noun)
            
tm_nouns[0:10]

['배송', '전자', '제품', '시간', '배송', '사진', '안', '제품', '문제', '배송']

# 감성 분석(Sentiment Analysis)



# 한국어 감정분석

### 한국어 감정어 모음인 word_info.json 파일을 하위폴더인 data폴더에 미리 다운받아 놓는다.

In [25]:
def data_list(wordname):

    with open('C:/Users/smell/OneDrive/바탕 화면/tool 모음/수업/2학기 수업/파이썬 텍스트 마이닝/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:

        data = json.load(f)
    result = ['None','3']

    for i in range(0, len(data)):
        if data[i]['word'] == wordname:
            result.pop()
            result.pop()
            result.append(data[i]['word_root'])
            result.append(data[i]['polarity'])

    r_word = result[0]
    s_word = result[1]

    return [r_word, s_word]

#     print("-2:매우 부정, -1:부정, 0:중립 or Unkwon, 1:긍정, 2:매우 긍정")

In [26]:
tm_morphs = [] 
for post in tm_posts:
    posts = []
    if type(post) == str:
        for morphs in tagger.morphs(post):
            if morphs not in tm_stop_words:
                posts.append(morphs)
    posts = ' '.join(posts)
    tm_morphs.append(posts)

tm_morphs[0:3]

['잘 받 았 습니다 ! 배송 빠르 지 않 았 지만 전자 제품 시간 걸리 니 많이 늦 지 않 았 습니다 그런데 배송 받 고 보 니 . .. 사진 처럼 부딪히 고 찢어진 들 보여서 ! 안 제품 에 문제 잘 받 았 습니다 ! 배송 빠르 지 않 았 지만 전자 제품 시간 걸리 니 많이 늦 지 않 았 습니다 그런데 배송 받 고 보 니 . .. 사진 처럼 부딪히 고 찢어진 들 보여서 ! 안 제품 에 문제 없 나 생각 했 는데 ! 오늘 만두 넣 고 돌려 보 니 아주 잘 됩니다 안 스티로폼 으로 쌓여 있 어서 제품 멀쩡 했 습니다 충격 받 았 을 도 같 습니다 . 조심히 배송 이나 관리 되 면 좋 겠 습니다 . 인터넷 에 찾 다가 여기 가격 싸 에 얼른 구입 했 고 , 좋 습니다 ! ! 참 , 제조 일자 2020 년 2 월 입니다 !',
 '제품 자체 디자인 기능 가격 모두 마음 에 듭니다만 . ... 오븐 에 어 프라이 기능 과 합쳐진 멀티 제품 으로 생각 바뀌 되 어 . .. 반품 하 려고 했으니 주일 지나 버렸 네요 . . 지인 한테 넘기 던지 제품 자체 디자인 기능 가격 모두 마음 에 듭니다만 . ... 오븐 에 어 프라이 기능 과 합쳐진 멀티 제품 으로 생각 바뀌 되 어 . .. 반품 하 려고 했으니 주일 지나 버렸 네요 . . 지인 한테 넘기 던지 하 고 멀티 제품 으로 야 할 같 네요 하지만 가격 에 사용 편리 하 고 제품 외관 까지 생각 해 주 셔서 감사 합 니 닷 . . 멀티 제품 도 sk 매직 으로 생각 하 고 있 어요',
 '포장 잘 해서 적당히 도착 했 는데 제풍 라고 불량 제품 표시 한 부분 과 그 위 에 일자 로 길 , 긴 부분 사진 으론 안 보여 촬영 포기 녹 난 같 은데 그냥 동장 해서 조립 판매 . .. 이건 포장 잘 해서 적당히 도착 했 는데 제풍 라고 불량 제품 표시 한 부분 과 그 위 에 일자 로 길 , 긴 부분 사진 으론 안 보여 촬영 포기 녹 난 같 은데 그냥 동장 해서 조립 판매 . .. 이건 아닌 같 데 하 면서 도 니까 그냥 쓰 

In [27]:
import json
from tqdm import tqdm

sentiment_firm=[]

for tm_words in tqdm(tm_morphs):
    sentiment_firm_line = []
    sentiment = 0
    count = 0
    
    for i in range(len(tm_words)):
        emotion_an = data_list(tm_words[i])
        if emotion_an[1] == '3':
            pass
        else:
            sentiment += int(emotion_an[1])
            count += 1
    try:
        sentiment_firm.append(sentiment/count)
    except:
        sentiment_firm.append(0)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [20:20<00:00,  1.22s/it]


In [30]:
kor_sent=pd.DataFrame(sentiment_firm)

kor_sent

,0
0,1.000000
1,-2.000000
2,-1.000000
3,0.000000
4,0.571429
...,...
995,0.000000
996,0.000000
997,0.000000
998,-2.000000


In [31]:
# # 감성분석한 쇼핑후기 데이터 xlsx로 저장하기
kor_sentiment = pd.ExcelWriter('C:/Users/smell/OneDrive/바탕 화면/tool 모음/수업/2학기 수업/파이썬 텍스트 마이닝/skm_blog_sentiment_shop.xlsx', engine='xlsxwriter')
kor_sent.to_excel(kor_sentiment, sheet_name='Sheet1')
kor_sentiment.close()